In [1]:
# Pythonで訓練をする
# deviceはcuda or cpu or mps
#  mps: apple silicon
# out of memoryが発生した際には、batch_sizeを減らす


# 途中から計算するときには，modelディレクトリに checkpoint_xxx.pt(xxxは計算済みのepoch数)とcheckpoint_best.pt が存在すること

In [2]:
!python ./model.py \
  --emb_size 512 \
  --nhead 8 \
  --ffn_hid_dim 1024 \
  --batch_size 32 \
  --num_encoder_layers 8 \
  --num_decoder_layers 8 \
  --lr 0.00002 \
  --dropout 0.3 \
  --num_epochs 100 \
  --device cuda \
  --earlystop_patient 3 \
  --output_dir model \
  --tensorboard_logdir logs \
  --prefix translation \
  --source_lang eng \
  --target_lang fra \
  --train_file ../dataset/train.jsonl \
  --valid_file ../dataset/val.jsonl

num_epochs:100
Epoch: 1, Train loss: 3.089, Val loss: 2.581, Epoch time = 657.318s
Epoch: 2, Train loss: 2.565, Val loss: 2.408, Epoch time = 656.787s
Epoch: 3, Train loss: 2.429, Val loss: 2.318, Epoch time = 655.462s
Epoch: 4, Train loss: 2.352, Val loss: 2.247, Epoch time = 656.760s
Epoch: 5, Train loss: 2.294, Val loss: 2.200, Epoch time = 656.241s
Epoch: 6, Train loss: 2.248, Val loss: 2.145, Epoch time = 655.842s
Epoch: 7, Train loss: 2.209, Val loss: 2.109, Epoch time = 654.220s
Epoch: 8, Train loss: 2.175, Val loss: 2.071, Epoch time = 657.698s
Epoch: 9, Train loss: 2.145, Val loss: 2.030, Epoch time = 655.461s
Epoch: 10, Train loss: 2.116, Val loss: 2.006, Epoch time = 654.626s
Epoch: 11, Train loss: 2.091, Val loss: 1.973, Epoch time = 654.517s
Epoch: 12, Train loss: 2.067, Val loss: 1.945, Epoch time = 654.999s
Epoch: 13, Train loss: 2.045, Val loss: 1.921, Epoch time = 655.722s
Epoch: 14, Train loss: 2.023, Val loss: 1.898, Epoch time = 655.175s
Epoch: 15, Train loss: 2.002

In [ ]:
# javaのDJLで使えるようにモデルファイルを変換する

In [2]:
!python convert.py  \
    --model_file=model/checkpoint_best.pt \
    --model_script=model/script.pt \
    --encoder=model/encoder.pt \
    --decoder=model/decoder.pt \
    --positional_encoding=model/positional_encoding.pt \
    --generator=model/generator.pt \
    --src_tok_emb=model/src_tok_emb.pt \
    --tgt_tok_emb=model/tgt_tok_emb.pt \
    --vocab_src=model/vocab_src.txt \
    --vocab_tgt=model/vocab_tgt.txt \
    --params=model/params.json \
    --device=cpu
